# Flappy Bird Junco Sim
2D flight sim for EECE5500 Mobile Robotics

In [2]:
############################### Import libraries ###############################
import os
import glob
import time
from datetime import datetime

from PIL import Image

import matplotlib.pyplot as plt
%matplotlib qt

import numpy as np
from numpy import matlib
import scipy as sci

import gym
import control
import ConvexMotionPlanning
from rrtplanner import RRTStar

In [3]:
class Drone():
    dt = 0.01
    m = 3.2
    Iyy = 1/12*m*0.8**2

    S = 0.25
    c = 0.13

    Cl_0 = 0.5
    Cl_alpha = 0.1*180/np.pi

    Cd_0 = 0.1
    K = 0.05

    Cm_0 = 0.5
    Cm_alpha = -0.14*180/np.pi
    Cm_alpha_dot = -0.008*180/np.pi
    Cm_delta_e = 0.2

    g = 9.81

    vNom = 20
    rhoNom = 1.225
    qNom = 0.5*rhoNom*vNom**2

    ANom = np.array([[0, 1], [Cm_alpha*qNom*S/Iyy, Cm_alpha_dot*qNom*S/Iyy]])
    BNom = np.array([[0], [qNom*Cm_delta_e*S]])
    Q =  np.array([[1, 0], [0, 10]])
    R =  np.array([[0.5]])
    Kgain = control.lqr(ANom, BNom, Q, R)[0]
    print(Kgain)

    KFull = np.zeros(4)

    Qestimation = np.eye(4)/100
    Pestimation = np.eye(4)/100
    Restimation = np.array([[1, 0], [0, (0.5/180*np.pi)**2]])
    stateEstimate = np.array([0.0,0.0,0.0,0.0])

    lidar_range = 50
    lidar_angle = 100*np.pi/180
    lidar_res = int(lidar_angle*180/np.pi)
    lidar = [lidar_range, lidar_angle, lidar_res]
    
    def __init__(self):
        self.x_thres = 10000
        self.z_thresHigh = 500
        self.z_thresLow = 0
        self.v_thres = 50
        self.theta_thres = np.pi/6
        self.theta_dot_thres = np.pi
        self.gamma_thres = np.pi/6
        self.grid = []
        # ???
        self.delta_e_stale = 0
        self.thrust_stale = 0


        self.limits = np.array([
            self.x_thres,
            self.z_thresHigh,
            self.z_thresLow,
            self.v_thres,
            self.theta_thres,
            self.theta_dot_thres,
            self.gamma_thres
        ])

        self.state = None
        self.prev_action = None
        self.steps_beyond_terminated = None
        self.time = 0.

        self.objects = None

    def thetaFromElevator(self, T,thetaDes, delta_e):
        alphaNom = -(self.Cm_0+self.Cm_delta_e*delta_e)/self.Cm_alpha
        Clnom = self.Cl_0 + self.Cl_alpha*alphaNom
        Cdnom = self.Cd_0 + self.K*Clnom**2
        return thetaDes- (-Cdnom/Clnom + T/(self.m*self.g)*(1-Cdnom/Clnom*alphaNom))-alphaNom

    def vGammaFromElevator(self, T, delta_e):
        alphaNom = -(self.Cm_0+self.Cm_delta_e*delta_e)/self.Cm_alpha
        Clnom = self.Cl_0 + self.Cl_alpha*alphaNom
        Cdnom = self.Cd_0 + self.K*Clnom**2
        gammaNom = Cdnom/Clnom*(1-T/self.m/self.g*np.sin(alphaNom)) + T*np.cos(alphaNom)/self.m/self.g
        vNom = np.sqrt(2*(-T*np.sin(alphaNom)+self.m*self.g*np.cos(gammaNom))/(self.S*self.rhoNom))
        return vNom, gammaNom

    def solveForElevator(self, T, thetaDes):
        func = lambda x: self.thetaFromElivator(T, thetaDes,x)
        sol = sci.optimize.root_scalar(func, method='secant', x0 = 0, x1 = 1)
        return sol.root
    
    def elevatorFromAlpha(self, alpha):
        return -(self.Cm_alpha*alpha+self.Cm_0)/self.Cm_delta_e 

    # Calculates a "coherent" (steady state stable) command from thrust and gamma command
    # Returns airspeed, theta, thetadot, and flight path angle (gamma)
    def coherentCommand(self, T, gamma):
        airspeed = self.vNom
        theta = 0
        thetaDot = 0
        verticalForces = lambda X: ((self.Cl_0 + self.Cl_alpha*(X[0]- gamma))*1/2*(X[1]**2)*self.rhoNom*self.S - self.m*self.g*np.cos(gamma) + T*np.sin(X[0]-gamma), \
               ((self.Cd_0+self.K*(self.Cl_0 + self.Cl_alpha*(X[0]- gamma))**2)*1/2*(X[1]**2)*self.rhoNom*self.S -T*np.cos(X[0]-gamma)+self.m*self.g*np.sin(gamma)))

        sol = sci.optimize.root(verticalForces, x0 = [theta, airspeed])
        X =sol.x      
        return [sol.x[1], sol.x[0], thetaDot, gamma]

    # Returns a continous time Jacobian for state and control
    def calculateCTSABMatrix(self, stateIn):
        v, theta, theta_dot, gamma = stateIn
        q = 0.5*self.rhoNom*v**2
        alpha = theta-gamma

        #Calculate alphaDot
        Cl = self.Cl_0 + self.Cl_alpha*alpha
        L = q*self.S*Cl
        gamma_dot = (L - self.m*self.g*np.cos(gamma) +self.thrust_stale*np.sin(alpha)) / (self.m*v)
        alpha_dot = theta_dot-gamma_dot

        #Other aero
        Cd = self.Cd_0 + self.K*Cl**2
        Cm = self.Cm_0 + self.Cm_alpha*alpha + self.Cm_alpha_dot*alpha_dot + self.Cm_delta_e*self.delta_e_stale
        D = q*self.S*Cd
        M = q*self.S*Cm

        #derivatives
        v_dot = (-D - self.m*self.g*np.sin(gamma) + self.thrust_stale*np.cos(alpha)) / self.m
        #theta_dot = theta_dot
        theta_ddot = M / self.Iyy
        gamma_dot = (L - self.m*self.g*np.cos(gamma) +self.thrust_stale*np.sin(alpha)) / (self.m*v)

        #Partial derivatives of aeroforces
        pDpv = self.rhoNom*v*self.S*Cd
        pDptheta = 2*self.K*q*self.S*(self.Cl_alpha * self.Cl_0 + self.Cl_alpha**2*alpha)
        pDpthetaDot = 0
        pDpgamma = -2*self.K*q*self.S*(self.Cl_alpha+self.Cl_alpha**2*alpha)
        pMpv = self.rhoNom*v*self.S*Cm
        pMptheta = q*self.S*self.Cm_alpha
        pMpthetaDot = 0
        pMpgamma = -q*self.S*self.Cm_alpha
        pLpv = self.rhoNom*v*self.S*Cl
        pLptheta = q*self.Cl_alpha
        pLpthetaDot = 0
        pLpgamma = -q*self.Cl_alpha

        #out of order entries
        pf4ptheta = (pLptheta + self.thrust_stale*np.cos(alpha))/(self.m*v)
        pf4pgamma = (pLpgamma/self.m +self.g*np.sin(gamma)-self.thrust_stale/self.m*np.cos(alpha))/v

        #Partial derivatives of nonlinear map with respect to tstate
        pf1pv = - 1/self.m*pDpv
        pf1ptheta = (-pDptheta-self.thrust_stale*np.sin(alpha))/self.m
        pf1pthetaDot = 0
        pf1pgamma = (-pDpgamma/self.m - self.g*np.cos(gamma) + self.thrust_stale/self.m*np.sin(alpha))
        pf2pv = 0
        pf2ptheta = 0
        pf2pthetaDot = 1
        pf2pgamma = 0
        pf3pv = pMpv/self.Iyy
        pf3ptheta = pMptheta/self.Iyy
        pf3pthetaDot = 0
        pf3pgamma = pMpgamma/self.Iyy
        pf4pv = pLpv/(self.m*v)
        pf4pthetaDot = 0

        AFull = np.array([[pf1pv, pf1ptheta, pf1pthetaDot, pf1pgamma],[pf2pv, pf2ptheta, pf2pthetaDot, pf2pgamma],
                      [pf3pv, pf3ptheta, pf3pthetaDot, pf3pgamma],[pf4pv, pf4ptheta, pf4pthetaDot, pf4pgamma]])
        BFull = np.array([[0], [0], [q*self.Cm_delta_e*self.S],[0]])
        return AFull, BFull

    def calculateGains(self):
        x, y, v, theta, theta_dot, gamma = self.state
        AFull, BFull =  self.calculateCTSABMatrix(self.state[2:])

        Qcontrol =  np.array([[0.1,0,0,0],[0,1, 0,0], [0,0, 10,0], [0, 0, 0, 20]])
        Rcontrol =  np.array([[1]])
        Ncontrol = np.zeros([4, 1])
        H = np.array([[1,0,0,0],[0,1,0,0]])

        self.KFull = control.lqr(AFull, BFull, Qcontrol, Rcontrol)[0]

    def EKF(self):
        v, theta, theta_dot, gamma = self.stateEstimate
        alpha = theta-gamma
        q = 0.5*self.rhoNom*v**2

        #Calculate alphaDot
        Cl = self.Cl_0 + self.Cl_alpha*alpha
        L = q*self.S*Cl
        gamma_dot = (L - self.m*self.g*np.cos(gamma) +self.thrust_stale*np.sin(alpha)) / (self.m*v)
        alpha_dot = theta_dot-gamma_dot

        #Other aero
        Cd = self.Cd_0 + self.K*Cl**2
        Cm = self.Cm_0 + self.Cm_alpha*alpha + self.Cm_alpha_dot*alpha_dot + self.Cm_delta_e*self.delta_e_stale
        D = q*self.S*Cd
        M = q*self.S*Cm

        #derivatives
        v_dot = (-D - self.m*self.g*np.sin(gamma) + self.thrust_stale*np.cos(alpha)) / self.m
        #theta_dot = theta_dot
        theta_ddot = M / self.Iyy
        gamma_dot = (L - self.m*self.g*np.cos(gamma) +self.thrust_stale*np.sin(alpha)) / (self.m*v)

        AFull, BFull = self.calculateCTSABMatrix([v, theta, theta_dot, gamma])
        F = AFull*self.dt + np.eye(4) 

        #Observability 
        h = np.array([[self.state[2]],[self.state[3]]])
        H = np.array([[1.0,0.0,0.0,0.0],[0.0,1.0,0.0,0.0]])

        #predict
        v += v_dot*self.dt
        theta += theta_dot*self.dt 
        theta_dot += theta_ddot * self.dt
        gamma += gamma_dot*self.dt
        xEst = np.array([[v], [theta], [theta_dot], [gamma]])

        self.Pestimation = (F)@self.Pestimation @ np.transpose(F) + self.Qestimation
        #update
        y = np.array([[v],[theta]]) - np.array([[self.state[2]],[self.state[3]]]) + np.array([np.random.multivariate_normal([0,0], self.Restimation)]).T
        S = H @ self.Pestimation @ H.T + self.Restimation
        K = self.Pestimation @ np.transpose(H) @ sci.linalg.inv(S)
        xEst -= K @ y
        self.Pestimation = (np.eye(4) - K @ H) @ self.Pestimation
        self.stateEstimate = xEst.flatten()
    
    def step(self, action):

        def update_plot(ax,state,lidar,objects,beams=False):

            # filter objects to ones in lidar cone
            def seenObjects(state,angle,beam,objects):

                x,y,theta = state

                # objects within range
                idx = np.logical_and(
                    (objects[0,:]>= x),
                    (np.sqrt((objects[0,:]-x)**2+(objects[1,:]-y)**2) <= beam)
                )

                # above and below lines
                upper = np.tan(theta+angle/2)
                lower = np.tan(theta-angle/2)
                ind = np.arange(len(idx))
                for i in ind:
                    if idx[i]:
                        dx = objects[0,i] - x
                        if y+upper*dx < objects[1,i] or y+lower*dx > objects[1,i]:
                            idx[i] = False

                return objects[:,idx]

            # intersection of line and circle
            def intersect(p1,p2,obj):
                x1 = p1[0] - obj[0]
                x2 = p2[0] - obj[0]
                y1 = p1[1] - obj[1]
                y2 = p2[1] - obj[1]

                r = obj[2]

                dx = x2-x1
                dy = y2-y1
                dr = np.sqrt(dx*dx + dy*dy)
                D = x1*y2 - x2*y1

                cross = False
                disc = r**2*dr**2 - D**2
                if disc>=0:
                    cross = True
                    xp = (D*dy + np.sign(dy)*dx*np.sqrt(r**2*dr**2 - D**2)) / dr**2
                    xm = (D*dy - np.sign(dy)*dx*np.sqrt(r**2*dr**2 - D**2)) / dr**2
                    yp = (-D*dx + np.abs(dy)*np.sqrt(r**2*dr**2 - D**2)) / dr**2
                    ym = (-D*dx - np.abs(dy)*np.sqrt(r**2*dr**2 - D**2)) / dr**2
                    
                    dp = np.sqrt( (xp+obj[0])**2 + (yp+obj[1])**2 )
                    dm = np.sqrt( (xm+obj[0])**2 + (ym+obj[1])**2 )
                    if dp >= dm:
                        p2 = (xm+obj[0],ym+obj[1])
                    else:
                        p2 = (xp+obj[0],yp+obj[1])

                return p2, cross
            
            if self.animate and round(self.time,1)%.1==0:
                ax.clear()
            x,z,_,theta,_,_ = state
            beam,angle,res = lidar
            
            rays = np.zeros((2,res))
            if beams:
                objects = seenObjects([x,z,theta],angle,beam,objects)
                
                angles = np.linspace(-angle/2,angle/2,num=res)
                for i,a in enumerate(angles):
                    x1,y1 = x,z
                    x2,y2 = x+beam*np.cos(theta+a),z+beam*np.sin(theta+a)
                    
                    # check for lidar intersection
                    for obj in objects.T:
                        p,cross = intersect((x1,y1),(x2,y2),obj)
                        if cross:
                            if rays[0,i] == 0 or np.linalg.norm(rays[:,i]) > np.sqrt((p[0]-x1)**2+(p[1]-y1)**2):
                                x2,y2 = p
                                rays[0,i] = x2-x1
                                rays[1,i] = y2-y1

                    if self.animate and round(self.time,1)%.1==0:
                        ax.plot([x1,x2],[y1,y2],'k-')

            else:
                if self.animate and round(self.time,1)%.1==0:
                    ax.plot([x,x+beam*np.cos(theta+angle/2)],[z,z+beam*np.sin(theta+angle/2)],'k--')
                    ax.plot([x,x+beam*np.cos(theta-angle/2)],[z,z+beam*np.sin(theta-angle/2)],'k--')
                    a = np.linspace(-angle/2,angle/2,int(res/3))
                    xs = x+beam*np.cos(theta+a)
                    zs = z+beam*np.sin(theta+a)
                    ax.plot(xs,zs,'k.',markersize=1)

                    xs = objects[0,:]
                    ys = objects[1,:]
                    s = objects[2,:]
                    ax.scatter(xs,ys,50*s,'r')

            if self.animate and round(self.time,1)%.1==0:
                ax.set_ylabel('Height (m)')
                ax.set_xlim([x-0.5*beam,x+2.5*beam])
                ax.set_ylim([z-0.75*beam,z+0.75*beam])
                ax.set_aspect('equal')

            return rays

        def update_grid(ax,lidar,rays,res=1):

            # returns the neighbors of a vertex v in G
            def neighbors(v,G):
                
                # initial neighbors
                i,j = v
                row = np.array([i-1,i-1,i-1,i,i,i+1,i+1,i+1])
                col = np.array([j-1,j,j+1,j-1,j+1,j-1,j,j+1])

                # remove cells out of range
                r,c = np.shape(G)
                low = np.logical_and(row>=0, col>=0)
                high = np.logical_and(row<=r, col<=c)
                idx = np.logical_and(low, high)
                row = np.delete(row,~idx)
                col = np.delete(col,~idx)

                return list(zip(row,col))
            
            beam,_,_ = lidar
            r,c = int(1.5*beam/res), int(3*beam/res)
            grid = np.zeros((r,c))
            x,y = int(c/6),int(r/2)
                        
            grid[y,x] = 1

            r,c = np.shape(rays)
            for i in range(c):
                if rays[0,i] != 0:
                    grid[(y+rays[1,i]).astype(int),(x+rays[0,i]).astype(int)] = 1
            
            if self.animate and round(self.time,1)%.1==0:
                ax.clear()
                ax.imshow(grid,cmap='gray')
                ax.invert_yaxis()
                ax.set_aspect('equal')
                ax.set_yticklabels([])
                ax.set_xticklabels([])
                ax.set_xlabel('Distance (m)')
                ax.set_ylabel('Height (m)')

            grid[y,x] = 0
            padded = grid.copy()
            r,c = np.shape(grid)
            for i in range(r):
                for j in range(c):
                    if grid[i,j]:
                        n = neighbors((i,j),grid)
                        for k in n:
                            padded[k] = 1
        
            return padded
        err_msg = f"{action!r} ({type(action)}) invalid"
        assert self.state is not None, "Call reset before using step method."
        
        #Extra states
        x, z, v, theta, theta_dot, gamma = self.state
        rho = np.random.normal(self.rhoNom,0.0)
        alpha = theta - gamma
        q = 0.5*rho*v**2
        
        #Control inputs
        if (self.time > 0):
            self.EKF()
        thrust, stateCommand, delta_e = action

        #delta_e = self.elevatorFromAlpha(stateCommand[1]-stateCommand[3])
        controlDirect = self.Kgain @ (np.array([[theta], [theta_dot]])-np.array([[stateCommand[1]], [0]]))
        error = (np.array([self.stateEstimate]).T-np.array([stateCommand]).T)
        controlFull = self.KFull @ error
        delta_e = delta_e-controlFull[0][0]

        #alpha dot
        Cl = self.Cl_0 + self.Cl_alpha*alpha
        L = q*self.S*Cl
        gamma_dot = (L - self.m*self.g*np.cos(gamma) +thrust*np.sin(alpha)) / (self.m*v)
        alpha_dot = theta_dot-gamma_dot

        # Forces
        Cd = self.Cd_0 + self.K*Cl**2
        Cm = self.Cm_0 + self.Cm_alpha*alpha + self.Cm_alpha_dot*alpha_dot + self.Cm_delta_e*delta_e
        D = q*self.S*Cd
        M = q*self.S*Cm

        #derivatives
        x_dot = v*np.cos(gamma)
        z_dot = v*np.sin(gamma)
        v_dot = (-D - self.m*self.g*np.sin(gamma) + thrust*np.cos(alpha)) / self.m
        #theta_dot = theta_dot
        theta_ddot = M / self.Iyy
        #Old controls for filter
        self.delta_e_stale = delta_e
        self.thrust_stale = thrust

        # integrate
        x += x_dot*self.dt
        z += z_dot*self.dt
        v += v_dot*self.dt 
        theta += theta_dot*self.dt
        theta_dot += theta_ddot*self.dt
        gamma += gamma_dot*self.dt

        state = np.array([x,z,v,theta,theta_dot,gamma])
        action = np.array([thrust,delta_e])

        self.state = (list(np.reshape(state,(6,))))
        self.time += self.dt
        terminated = bool(
            x < -self.x_thres
            or x > self.x_thres
            or z > self.z_thresHigh
            or z < self.z_thresLow
            or v < -self.v_thres
            or v > self.v_thres
            or theta < -self.theta_thres
            or theta > self.theta_thres
            or theta_dot < -self.theta_dot_thres
            or theta_dot > self.theta_dot_thres
            or gamma < -self.gamma_thres
            or gamma > self.gamma_thres
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            self.steps_beyond_terminated = 0
            reward = 0.0
        else:
            if self.steps_beyond_terminated == 0:
                gym.logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0.0

        update_plot(self.axs[0],state,self.lidar,self.objects)
        rays = update_plot(self.axs[1],state,self.lidar,self.objects,beams=True)
        self.grid = update_grid(self.axs[2],self.lidar,rays)

        if self.animate and round(self.time,2)%.1==0:
            self.axs[0].set_title('Ground Truth')
            self.axs[1].set_title('Lidar View')
            self.axs[2].set_title('Occupancy Grid')
            plt.show()
            plt.pause(0.01)

        return state, reward, terminated, rays.astype(int)

    def reset(self,animate=True):
        self.state = np.array([0.0,100.0,self.vNom,0.0,0.0,0.0])
        self.state[2:] = self.coherentCommand(4,0/180*np.pi)
        self.stateEstimate = self.coherentCommand(4,0/180*np.pi)
        self.Pestimation = self.Qestimation.copy()
        self.steps_beyond_terminated = None
        self.time = 0
        self.calculateGains()

        self.animate = animate
        %matplotlib qt
        plt.close()
        self.fig,self.axs = plt.subplots(3,1,figsize=(6,10))

        num = 8
        x_pos = np.linspace(self.state[0]+30,self.state[0]+240,num)
        y_pos = np.random.uniform(low=self.state[1]-0.5,high=self.state[1]+0.5,size=num)
        sizes = np.random.uniform(low=0.5,high=0.5,size=num)
        self.objects = np.stack((x_pos,y_pos,sizes))

        return self.state

    def close(self):
        pass

[[0.00425524 0.72512213]]


In [86]:

np.set_printoptions(threshold=np.inf)
env = Drone()
state = env.reset(animate=False)
iterations = 2000
traj = np.zeros((7,iterations))
trajEst = np.zeros((5,iterations))
thrustCommand = np.zeros(iterations)+5

for iter in range(0, iterations):
    action = [thrustCommand[iter], env.coherentCommand(thrustCommand[iter],1/180*np.pi)] 
    state, reward, done, grid = env.step(action)
    traj[:,iter] = np.hstack((state,env.time))
    trajEst[:, iter] = np.hstack((env.stateEstimate,env.time))
    if done:
        
        break

r,c = np.shape(grid)
for i in range(r-1):
    for j in range(c-1):
        if grid[i,j]:
            plt.scatter(i,j)
plt.show()

ValueError: not enough values to unpack (expected 3, got 2)

In [102]:
import importlib
from rrtplanner import plot_rrt_lines, plot_path, plot_og, plot_start_goal
importlib.reload(ConvexMotionPlanning)
np.set_printoptions(threshold=np.inf)
env = Drone()
state = env.reset(animate=True)
iterations = 400
rrtPlot = False
traj = np.zeros((7,iterations))
commandLen = 400
thrustCommandVal = 4
thrustCommand = np.zeros(iterations)+thrustCommandVal
stateCommand = env.coherentCommand(thrustCommand[0],0/180*np.pi)
elivCommandRef = env.elevatorFromAlpha(stateCommand[1]-stateCommand[3])
elivCommand = np.zeros(iterations) + elivCommandRef
xCommand = np.zeros(iterations)
yCommand = np.zeros(iterations)
elvActual = np.zeros(iterations)

refCommand = np.zeros((iterations, 4))
for i in range(len(thrustCommand)):
    refCommand[i,:] = np.array(env.coherentCommand(thrustCommand[i],0/180*np.pi))
thrustCommandRef = thrustCommand[0]

pathDist = 400
for iter in range(0, iterations):
    action = [thrustCommand[iter], refCommand[iter, :], elivCommand[iter]] 
    state, reward, done, grid = env.step(action)
    traj[:,iter] = np.hstack((state,env.time))
    elvActual[iter] = env.delta_e_stale

    if(iter%pathDist == 0):
        #Produces an error when I don't hard code these
        midpoint =38
        lookAhead = 48
        orignX = 14
        xstart = np.array([midpoint, orignX]) 
        xgoal = np.array([midpoint, orignX+lookAhead])
        n = 3000
        r_rewire = 8
        rrts = RRTStar(env.grid[:, 0:70], n, r_rewire, pbar = False) 
        T, gv = rrts.plan(xstart, xgoal)
        path = rrts.route2gv(T, gv)
        path_pts = rrts.vertices_as_ndarray(T, path)
        tEnd = commandLen
        if(rrtPlot):
            fig = plt.figure()
            ax = fig.add_subplot()

            #these functions alter ax in-place.
            plot_og(ax, env.grid)
            plot_start_goal(ax, xstart, xgoal)
            plot_rrt_lines(ax, T)
            plot_path(ax, path_pts)
        refStates = env.coherentCommand(thrustCommandVal,thrustCommandVal/180*np.pi)
        referencePoints, referenceVels= ConvexMotionPlanning.calculateReferencePoints(tEnd+1, path_pts)
        referencePointsDyn, referenceVelsDyn = ConvexMotionPlanning.calculateReferencePoints(tEnd+1, path_pts)

        for i in range(len(referencePointsDyn)):
            referencePointsDyn[i][0] = referencePointsDyn[i][0]-refStates[0]*np.sin(0)*i*env.dt-midpoint
            referencePointsDyn[i][1] = referencePointsDyn[i][1]-refStates[0]*np.cos(0)*i*env.dt-orignX

        Acts, Bcts = env.calculateCTSABMatrix(refStates)
        Aopt = np.eye(Acts.shape[0]+2)
        Aopt[0, 2] = env.dt
        Aopt[1, 5] = refStates[0]*env.dt
        Aopt[2:,2:] = Aopt[2:,2:]+Acts*env.dt
        Bopt = np.zeros([6,2])
        Bopt[2:,0:1] = Bcts*env.dt
        alphaEst = refStates[1] - refStates[3]
        Bopt[2, 1] = 1/env.m*np.cos(alphaEst)*env.dt
        Bopt[5, 1] = 1/env.m*np.sin(alphaEst)/refStates[0]*env.dt

        xstart = np.concatenate(([0.,0.], np.array(env.stateEstimate) - np.array(refStates)))
        xgoal = np.concatenate(([0.,0.], [0.,0.,0.,0.]))
        xsol, usol = ConvexMotionPlanning.localTrajOpt(Aopt, Bopt, tEnd, env.grid, referencePoints, referencePointsDyn, xstart, xgoal)
        for i in range(pathDist):
            thrustCommand[iter + i] = usol[i][1] +thrustCommandVal 
            elivCommand[iter + i] = usol[i][0] + elivCommandRef
            xCommand[iter + i] = xsol[i][0]+refStates[0]*np.cos(0)*i*env.dt+env.state[0]
            yCommand[iter + i] = xsol[i][1]+refStates[0]*np.sin(0)*i*env.dt+env.state[1]
            refCommand[iter + i, :] = xsol[i][2:6].T + np.array(refStates)

    if done:

        break

r,c = np.shape(grid)
for i in range(r-1):
    for j in range(c-1):
        if grid[i,j]:
            plt.scatter(i,j)
plt.show()

     pcost       dcost       gap    pres   dres
 0:  8.2091e+02 -2.0021e+05  2e+05  2e-01  5e-06
 1:  2.5758e+02 -1.0454e+05  1e+05  6e-02  2e-06
 2:  2.3681e+02 -9.4431e+04  1e+05  4e-02  1e-06
 3:  2.7728e+02 -8.6594e+04  1e+05  4e-02  1e-06
 4:  4.6204e+02 -6.1892e+04  1e+05  4e-02  1e-06


KeyboardInterrupt: 

In [92]:
def state_plots(traj,control,xCommand, yCommand, elivCommand,elvActual, refCommand):

    def plot_state(ax,x,y,xlabel,ylabel):
        lim = 2
        ax.plot(x,y)
        ax.set_ylim([min(y)-lim,max(y)+lim])
        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)

        return
    vRef = refCommand[:,0]
    thetaRef = refCommand[:,1]
    gammaRef = refCommand[:,3]

    x = traj[0,:]
    z = traj[1,:]
    v = traj[2,:]
    theta = traj[3,:]
    thetaDot = traj[4,:]
    gamma = traj[5,:]
    time = traj[6,:]

    plt.close()
    fig, axs = plt.subplots(3,3, figsize=(8,8))
    plot_state(axs[0, 0],x,z,"Downrange distance (m)","Height (m)")
    plot_state(axs[0, 0],xCommand,yCommand,"Downrange distance (m)","Height (m)")
    axs[0,0].scatter(env.objects[0,:],env.objects[1,:], color = "red" )
    plot_state(axs[0, 1],time,(theta-gamma)*180/np.pi,"Time (s)","alpha (deg)")
    plot_state(axs[0, 1],time,(thetaRef-gammaRef)*180/np.pi,"Time (s)","alpha (deg)")
    plot_state(axs[0, 2],time,elvActual,"Time (s)","Elivator Actual")
    plot_state(axs[0, 2],time,elivCommand,"Time (s)","Elivator Command")
    plot_state(axs[1, 0],time,gamma*180/np.pi,"Time (s)","Gamma (deg)")
    plot_state(axs[1, 0],time,gammaRef*180/np.pi,"Time (s)","Gamma (deg)")
    plot_state(axs[1, 1],time,v,"Time (s)","Velocity (m/s)")
    plot_state(axs[1, 1],time,vRef,"Time (s)","Velocity (m/s)")
    plot_state(axs[2, 0],time,theta*180/np.pi,"Time (s)","Theta (deg)")
    plot_state(axs[2, 0],time,thetaRef*180/np.pi,"Time (s)","Theta (deg)")
    plot_state(axs[2, 1],time,control,"Time (s)","Thrust Command (N)")
    fig.show()
    return

state_plots(traj,thrustCommand, xCommand, yCommand, elivCommand, elvActual, refCommand)

In [ ]:
def est_plots(traj, trajEst,control):

    def plot_state(ax,x,y,xlabel,ylabel):
        lim = 2
        ax.plot(x,y)
        ax.set_ylim([min(y)-lim,max(y)+lim])
        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)
        return
    def plot_state_dashed(ax,x,y,xlabel,ylabel):
        lim = 2
        ax.plot(x,y,'--')
        ax.set_ylim([min(y)-lim,max(y)+lim])
        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)

        return
    
    x = traj[0,:]
    z = traj[1,:]
    v = traj[2,:]
    vEst =  trajEst[0,:]
    theta = traj[3,:]
    thetaEst = trajEst[1,:]
    thetaDot = traj[4,:]
    thetaDotEst = trajEst[2,:]
    gamma = traj[5,:]
    gammaEst = trajEst[3,:]
    time = traj[6,:]

    plt.close()
    fig, axs = plt.subplots(4,2, figsize=(8,1))
    plot_state(axs[0, 0],time,theta*180/np.pi,"Time (s)", "Theta (deg)")
    plot_state(axs[0, 1],time,thetaEst*180/np.pi,"Time (s)", "Theta Estimate (deg)")
    plot_state(axs[1, 0],time,thetaDot*180/np.pi,"Time (s)", "ThetaDot (deg/s)")
    plot_state(axs[1, 1],time,thetaDotEst*180/np.pi,"Time (s)", "ThetaDot Estimate (deg/s)")
    plot_state(axs[2, 0],time,v,"Time (s)", "Velocity (m/s)")
    plot_state(axs[2, 1],time,vEst,"Time (s)", "Velocity Estimate (m/s)")
    plot_state(axs[3, 0],time,gamma*180/np.pi,"Time (s)", "Gamma (deg)")
    plot_state(axs[3, 1],time,gammaEst*180/np.pi,"Time (s)", "Gamma Estimate (deg)")
    fig.show()
    return

est_plots(traj, trajEst, thrustCommand)

In [6]:
# Testing out ray tracing on the env.grid

#Basic sim setup
import importlib
from rrtplanner import plot_rrt_lines, plot_path, plot_og, plot_start_goal
importlib.reload(ConvexMotionPlanning)
np.set_printoptions(threshold=np.inf)
env = Drone()
state = env.reset(animate=True)
stateCommand = env.coherentCommand(4,0/180*np.pi)
elivCommandRef = env.elevatorFromAlpha(stateCommand[1]-stateCommand[3])

action = [4, stateCommand, elivCommandRef] 
state, reward, done, grid = env.step(action)
commandLen = 400

#Path planning
midpoint =37
lookAhead = 48
orignX = 25
xstart = np.array([midpoint, orignX]) 
xgoal = np.array([midpoint, orignX+lookAhead])
n = 3000
r_rewire = 5
rrts = RRTStar(env.grid[:, 0:80], n, r_rewire, pbar = False) 
T, gv = rrts.plan(xstart, xgoal)
path = rrts.route2gv(T, gv)
path_pts = rrts.vertices_as_ndarray(T, path)
tEnd = commandLen

referencePoints, referenceVels= ConvexMotionPlanning.calculateReferencePoints(tEnd+1, path_pts)

#Plot RRT
fig = plt.figure()
ax = fig.add_subplot()
print(env.grid.shape)
plot_og(ax, env.grid)
plot_start_goal(ax, xstart, xgoal)
plot_rrt_lines(ax, T)
plot_path(ax, path_pts)
ax.set_ylabel("Downrange distance in occupancy grid (m)")
ax.set_xlabel("Vertical distance in occupancy grid (m)")

#Generate search plot
numDirections = 16
directs = ConvexMotionPlanning.generateDirections(numDirections)

fig, ax = plt.subplots()
xRefsToPlot = list(zip(*referencePoints))[0]
yRefsToPlot = list(zip(*referencePoints))[1]

indexes = range(len(xRefsToPlot))
#indexes = indexes[0::75]
for i in indexes:
    cords = ConvexMotionPlanning.searchFromCord([xRefsToPlot[i],yRefsToPlot[i]], directs, env.grid)
    if (((np.sum(np.square((cords-np.array([xRefsToPlot[i],yRefsToPlot[i]]))), axis =1)) < 8**2).any()):
        plt.scatter(cords[:, 0], cords[:, 1], color = plt.cm.winter(i/np.max(indexes)))
        plt.scatter(xRefsToPlot[i], yRefsToPlot[i], color = "red")

    #plt.scatter(xRefsToPlot[i],yRefsToPlot[i], color = plt.cm.autumn(i/np.max(indexes)))
    #plt.axis('equal')
ax.imshow(env.grid[:, 0:80].T)
ax.invert_yaxis()
plt.plot(xRefsToPlot,yRefsToPlot, color = 'blue')
ax.set_ylabel("Downrange distance in occupancy grid (m)")
ax.set_xlabel("Vertical distance in occupancy grid (m)")
#ax.set(xlim=(0, 46), ylim = (14,60))


#In place plot
refStates = env.coherentCommand(4,0/180*np.pi)
referencePointsDyn, referenceVelsDyn = ConvexMotionPlanning.calculateReferencePoints(tEnd+1, path_pts)
for i in range(len(referencePointsDyn)):
    referencePointsDyn[i][0] = referencePointsDyn[i][0]-refStates[0]*np.sin(refStates[3])*i*env.dt-midpoint
    referencePointsDyn[i][1] = referencePointsDyn[i][1]-refStates[0]*np.cos(refStates[3])*i*env.dt-orignX

fig2, ax2 = plt.subplots()
xRefsToPlotDyn = list(zip(*referencePointsDyn))[0]
yRefsToPlotDyn = list(zip(*referencePointsDyn))[1]

indexes = range(len(xRefsToPlot))
#indexes = indexes[0::75]
for i in indexes:
    cords = ConvexMotionPlanning.searchFromCord([xRefsToPlot[i],yRefsToPlot[i]], directs, env.grid)
    if (((np.sum(np.square((cords-np.array([xRefsToPlot[i],yRefsToPlot[i]]))), axis =1)) < 8**2).any()):
        plt.scatter(cords[:, 1]-yRefsToPlot[i]+yRefsToPlotDyn[i], cords[:, 0]- xRefsToPlot[i]+xRefsToPlotDyn[i], color = plt.cm.winter(i/np.max(indexes)))
    #plt.scatter(xRefsToPlot[i],yRefsToPlot[i], color = plt.cm.autumn(i/np.max(indexes)))
        plt.axis('equal')

plt.plot(yRefsToPlotDyn,xRefsToPlotDyn, color = 'blue')
ax2.set_ylabel("Vertical distance, linearized frame (m)")
ax2.set_xlabel("Down range distance, linearized frame (m)")
#ax2.set(xlim=(-1, 40), ylim = (-1,100))
plt.show()



Acts, Bcts = env.calculateCTSABMatrix(refStates)
Aopt = np.eye(Acts.shape[0]+2)
Aopt[0, 2] = env.dt
Aopt[1, 5] = refStates[0]*env.dt
Aopt[2:,2:] = Aopt[2:,2:]+Acts*env.dt
Bopt = np.zeros([6,2])
Bopt[2:,0:1] = Bcts*env.dt
alphaEst = refStates[1] - refStates[3]
Bopt[2, 1] = 1/env.m*np.cos(alphaEst)*env.dt
Bopt[5, 1] = 1/env.m*np.sin(alphaEst)/refStates[0]*env.dt

xstart = np.concatenate(([0.,0.], np.array(env.stateEstimate) - np.array(refStates)))
xgoal = np.concatenate(([0.,0.], [0.,0.,0.,0.]))
xsol, usol = ConvexMotionPlanning.localTrajOpt(Aopt, Bopt, tEnd, env.grid, referencePoints, referencePointsDyn, xstart, xgoal)
thrustCommand = np.zeros(tEnd)
xCommand = np.zeros(tEnd)
yCommand = np.zeros(tEnd)
refCommand = np.zeros((tEnd, 4))
for i in range(tEnd):
    thrustCommand[i] = usol[i][1] + env.thrust_stale 
    xCommand[i] = xsol[i][0]+refStates[0]*np.cos(0)*i*env.dt+env.state[0]+orignX
    yCommand[i] = xsol[i][1]+refStates[0]*np.sin(0)*i*env.dt+env.state[1]+midpoint
    refCommand[i, :] = xsol[i][2:6].T + np.array(refStates)

ax.plot(yCommand-100,xCommand, color = 'black')


(75, 150)
     pcost       dcost       gap    pres   dres
 0:  2.3236e+03 -2.4851e+05  3e+05  4e-11  5e-06
 1:  4.9503e+02 -6.6585e+03  7e+03  4e-11  1e-07
 2:  2.3977e+02 -1.2569e+03  1e+03  9e-12  3e-08
 3:  2.4640e+02 -1.1742e+03  1e+03  7e-12  2e-08
 4:  2.1916e+02 -2.0449e+02  4e+02  2e-12  6e-09
 5:  2.1992e+02 -1.7915e+02  4e+02  2e-12  5e-09
 6:  2.1686e+02  1.6223e+02  5e+01  3e-13  2e-10
 7:  2.1610e+02  2.1533e+02  8e-01  5e-14  3e-12
 8:  2.1609e+02  2.1608e+02  8e-03  3e-15  2e-12
 9:  2.1609e+02  2.1609e+02  8e-05  3e-15  2e-12
Optimal solution found.
